In [268]:
import numpy as np

poly1 = [
    (0, -1),
    (-1, 0),
    (-1, 0),
    (0, 1),
]
poly2 = [
    (0.7071067811865476, 0.7071067811865476),
    (-0.7071067811865476, 0.7071067811865476),
    (-0.7071067811865476, -0.7071067811865476),
    (0.7071067811865476, -0.7071067811865476),
    (0.7071067811865476, -0.7071067811865476),
]

poly1 = list(set(poly1))
poly2 = list(set(poly2))

vertices = np.array(poly1)

In [269]:
vertices_roll = np.roll(vertices, (1,1), axis=(0,1))

prods = (vertices * vertices_roll).sum(axis=0)
area = 0.5 * abs(prods[0] - prods[1])
area

1.0

In [270]:
def area(vertices):
    # based on shoelace theorem: https://artofproblemsolving.com/wiki/index.php/Shoelace_Theorem
    vertices = np.array(vertices)
    vertices_rolled = np.roll(vertices, (-1,1), axis=(0,1))
    sum_of_prods = (vertices * vertices_rolled).sum(axis=0)
    return abs(sum_of_prods[0] - sum_of_prods[1])/2

In [271]:
poly_test  = [
    (3,4),
    (5,11),
    (12,8),
    (9,5),
    (5,6)
]
area(poly_test)

30.0

In [272]:
(3*11 + 5*8 + 12*5 + 9*6 + 5*4)

207

In [273]:
207-259

-52

In [274]:
import cmath

def angle(vertex):
    a, b = vertex
    phase = cmath.phase(complex(a,b))
    return phase + 2*cmath.pi if phase < 0 else phase

def angles(vertices):
    return {v: angle(v) for v in vertices} # change to for loop if oom

In [275]:
poly2 = [
    (0.7071067811865476, 0.7071067811865476),
    (-0.7071067811865476, 0.7071067811865476),
    (-0.7071067811865476, -0.7071067811865476),
    (0.7071067811865476, -0.7071067811865476),
    (0.7071067811865476, -0.7071067811865476),
]
angles(poly2)

{(0.7071067811865476, 0.7071067811865476): 0.7853981633974483,
 (-0.7071067811865476, 0.7071067811865476): 2.356194490192345,
 (-0.7071067811865476, -0.7071067811865476): 3.9269908169872414,
 (0.7071067811865476, -0.7071067811865476): 5.497787143782138}

In [276]:
def merge(l1, l2, key=lambda x:x):
    # l1, l2 lists such that `sorted(l1, key=key) == l1`. (resp. l2)
    p1 = 0
    p2 = 0
    out = []
    while p1 < len(l1) and p2 < len(l2):
        if key(l1[p1]) < key(l2[p2]):
            out.append(l1[p1])
            p1 += 1
        else:
            out.append(l2[p2])
            p2 += 1
    out.extend(l1[p1:]) # at least one extension will do nothing
    out.extend(l2[p2:])
    return out

In [277]:
a = [1,4,6,8,13]
b = [2,4,5,6,17]

merged = merge(poly1, poly2, key=angle)

In [278]:
def vertex_sources(poly0, poly1):
    sources = {}
    for v in poly0:
        sources[v] = set([1])
    for v in poly1:
        if v in sources:
            sources[v].add(2)
        else:
            sources[v] = set([2])
    return sources

sources = vertex_sources(poly1, poly2)

In [279]:
sources

{(0, 1): {1},
 (0, -1): {1},
 (-1, 0): {1},
 (0.7071067811865476, 0.7071067811865476): {2},
 (-0.7071067811865476, 0.7071067811865476): {2},
 (-0.7071067811865476, -0.7071067811865476): {2},
 (0.7071067811865476, -0.7071067811865476): {2}}

In [280]:
def reverse_index(vertices):
    return {v:i for i,v in enumerate(vertices)}

def slope(L):
    L = np.array(L)
    vec = L[1] - L[0]
    if vec[0] != 0:
        return vec[1]/vec[0]
    return vec[1] * math.inf

def intersection(L1, L2):
    if slope(L1) == slope(L2):
        raise Error('lines are parallel')
        
    x1,y1 = L1[0]
    x2, y2 = L1[1]
    x3, y3 = L2[0]
    x4, y4 = L2[1]

    m1 = slope(L1)
    m2 = slope(L2)
    b1 = -m1*x1 + y1
    b2 = -m2*x3 + y3
    
    if x1 == x2:
        return (x1, m2*x1 + b2)
    if x3 == x4:
        return (x3, m1*x3 + b1)
    
    inter_x = (b2 - b1)/(m1 - m2)
    inter_y = m1*inter_x + b1
    
    return inter_x, inter_y

L1 = poly1[0], poly1[1]
L2 = poly2[0], poly2[1]

In [281]:
intersection([(0,0),(1,1)], [(0,1), (1,0)])

(0.5, 0.5)

In [282]:
intersections = []

def switch(poly_curr, poly_prev):
    if len(poly_curr) == 2 and len(poly_curr) == 2:
        return True
    return len(poly_curr ^ poly_prev) > 0

poly1_rindex = reverse_index(poly1)
poly2_rindex = reverse_index(poly2)


polys = [poly1, poly2]
rindices = [poly1_rindex, poly2_rindex]
def rindex(vertex):
    if len(sources[vertex]) == 1:
        return rindices[list(sources[vertex])[0] - 1]
    
def poly(vertex):
    if len(sources[vertex]) == 1:
        return polys[list(sources[vertex])[0] - 1]

def pred(vertex):
    return poly(vertex)[rindex(vertex)[vertex] - 1]

def succ(vertex):
    return poly(vertex)[(rindex(vertex)[vertex] + 1)%len(poly(vertex))]

test = []
for i, v_curr in enumerate(merged):
    v_prev = merged[i-1] # negative indexing when i==0 intentional
    poly_curr, poly_prev = sources[v_curr], sources[v_prev]
    if switch(poly_curr, poly_prev):
        v_curr_pred = pred(v_curr)
        v_prev_succ = succ(v_prev)
        test.append([v_prev, v_prev_succ, v_curr, v_curr_pred])
        inter = intersection((v_curr, v_curr_pred), (v_prev, v_prev_succ))
        intersections.append(inter)

area(intersections)

/om2/user/rogerjin/conda/ganoli/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
/om2/user/rogerjin/conda/ganoli/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
/om2/user/rogerjin/conda/ganoli/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


0.6464466094067263

In [283]:
test

[[(0.7071067811865476, 0.7071067811865476),
  (-0.7071067811865476, 0.7071067811865476),
  (0, 1),
  (-1, 0)],
 [(0, 1),
  (0, -1),
  (-0.7071067811865476, 0.7071067811865476),
  (0.7071067811865476, 0.7071067811865476)],
 [(-0.7071067811865476, -0.7071067811865476),
  (0.7071067811865476, -0.7071067811865476),
  (0, -1),
  (0, 1)],
 [(-1, 0),
  (0, 1),
  (0.7071067811865476, -0.7071067811865476),
  (0.7071067811865476, -0.7071067811865476)]]

In [284]:
intersections

[(-0.2928932188134524, 0.7071067811865476),
 (0, 0.7071067811865476),
 (0, -0.7071067811865476),
 (0.7071067811865476, 1.7071067811865475)]

In [285]:
succ(poly1[0])

(0, -1)

In [286]:
succ(poly1[1])

(-1, 0)

In [287]:
succ(poly1[2])

(0, 1)

32.5